In [11]:
#import tempfile
import os
import datetime
import ocgis 
from ocgis.util.shp_process import ShpProcess
from ocgis.util.shp_cabinet import ShpCabinetIterator
from cdo import *   # python version
cdo = Cdo()

In [2]:
# define the environment
if 'obelix' in os.uname()[1] or 'asterix' in os.uname()[1]:
    HOME = os.getenv('HOME')
    OUT_PATH = '/home/estimr2/nhempelmann/data/extremoscop/'
    IN_DIR = '/home/estimr2/rvautard/BCCORDEX/DRIAS/V1/'
else: 
    HOME = os.getenv('HOME')
    OUT_DIR = HOME+'/data/extremoscop/'

In [3]:
# define ocgis environment

ocgis.env.OVERWRITE = True
ocgis.env.USE_CFUNITS = True
SHP_DIR = os.path.join(HOME + '/birdhouse/flyingpigeon/flyingpigeon/processes/shapefiles/FRA/')
ocgis.env.DIR_SHPCABINET = SHP_DIR
#ocgis.env.DIR_OUTPUT = OUT_DIR
sc = ocgis.ShpCabinet()
geoms = 'regions-2016'
sci = ShpCabinetIterator(geoms)

In [4]:
ref_19611900 = [datetime.datetime(1961,1,1),datetime.datetime(1990,12,31)]
ref_19712000 = [datetime.datetime(1971,1,1),datetime.datetime(2000,12,31)]
ref_19762005 = [datetime.datetime(1976,1,1),datetime.datetime(2005,12,31)]
vars = ['pr', 'tas', 'tasmin', 'tasmax']
indices = ['R20mm']

In [5]:
# get list of files
pr_files = os.listdir(IN_DIR +'pr/')
tas_files = os.listdir(IN_DIR +'tas/')
tasmin_files = os.listdir(IN_DIR +'tasmin/')
tasmax_files = os.listdir(IN_DIR +'tasmax/')

In [15]:
IN_DIR

'/home/estimr2/rvautard/BCCORDEX/DRIAS/V1/'

In [17]:
geom_select_sql_where = "'name' = 'Alsace, Champagne-Ardenne et Lorraine'"
geom_select_uid = [1]

In [31]:
## ocgis operation 
calc = [{'func': 'icclim_%s' %(indice), 'name': indice}]
calc_group = ['year'] # [[12, 1, 2], 'unique'

#select_ugid = [1]
for nc in pr_files:
    for indice in indices:
        for ugid in [1, 2, 3, 4, 5, 6, 7, 11,  13, 14, 15, 16, 17]: #  8, 9, 10, 12,
            rcp = nc.split('_')[1]
            prefix = nc.replace('pr', indice).replace('.nc', '')
            # create foulder structure and define output dir:
            
            if not os.path.exists(os.path.join(OUT_PATH , 'timeseries',  indice ,rcp , str(ugid))):
                os.makedirs(os.path.join(OUT_PATH , 'timeseries', indice ,rcp , str(ugid)))
            OUT_DIR = os.path.join(OUT_PATH , 'timeseries', indice ,rcp , str(ugid))
            
            if not os.path.exists(os.path.join(OUT_PATH , 'polygons',  indice ,rcp , str(ugid))):
                os.makedirs(os.path.join(OUT_PATH , 'polygons', indice ,rcp , str(ugid)))
            ocgis.env.DIR_OUTPUT = os.path.join(OUT_PATH , 'polygons', indice ,rcp , str(ugid))
            
            output = os.path.join(OUT_DIR,prefix+'.nc')
            
            if not os.path.exists(output):
                try:
                    if 'CM5A-MR_WRF331F' in nc: 
                        dimension_map = {'X': {'variable': 'lon', 'dimension': 'x', 'pos': 2},
                                    'Y': {'variable': 'lat', 'dimension': 'y', 'pos': 1},
                                    'T': {'variable': 'time', 'dimension': 'time', 'pos': 0}}
                    elif 'CNRM-CM5_CNRM-ALADIN53' in nc: 
                        dimension_map = {'X': {'variable': 'lon', 'dimension': 'x', 'pos': 2},
                                    'Y': {'variable': 'lat', 'dimension': 'y', 'pos': 1},
                                    'T': {'variable': 'time', 'dimension': 'time', 'pos': 0}}
                    elif 'MPI-ESM-LR_REMO019' in nc: 
                        dimension_map = {'X': {'variable': 'lon', 'dimension': 'x', 'pos': 2},
                                    'Y': {'variable': 'lat', 'dimension': 'y', 'pos': 1},
                                    'T': {'variable': 'time', 'dimension': 'time', 'pos': 0}}
                    else:     
                        dimension_map = {'X': {'variable': 'lon', 'dimension': 'x', 'pos': 2},
                                    'Y': {'variable': 'lat', 'dimension': 'y', 'pos': 1},
                                    'T': {'variable': 'time', 'dimension': 'time', 'pos': 0, 'bounds': 'time_bnds'}}

                    rd = ocgis.RequestDataset(os.path.join(IN_DIR, 'pr', nc), 'pr', dimension_map=dimension_map)

                    geom_nc = ocgis.OcgOperations(dataset=rd, geom=geoms, output_format='nc', 
                                                  select_ugid = ugid, prefix=prefix , 
                                                  add_auxiliary_files=False ,calc=calc, 
                                                  calc_grouping=calc_group).execute() #

                    cdo.fldmean (input = str(geom_nc) , output = output )

                    print ugid, prefix
                except Exception as e:
                    print 'failed for file : %s \n %s ' % (prefix, e )
            else: 
                print '%s \t %s already processed' % (ugid, prefix)

R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100 already processed
R20mm_rcp85_IPSL-IPSL-CM5A-MR_RCA4_1971-2100 already processed
R20mm_rcp85_IPSL-IPSL-CM5A-MR_RCA4_1971-2100 already processed
R20mm_rcp85_IPSL-IPSL-CM5A-MR_RCA4_1971-2100 already pr

In [ ]:
%bash 
ncview /home/estimr2/nhempelmann/data/extremoscop/R20mm_testfile.nc

In [76]:
for nc in pr_files:
    rd = ocgis.RequestDataset(os.path.join(IN_DIR, 'pr' ,nc) , 'pr')

In [77]:
os.path.join(IN_DIR,'pr',nc)

'/home/estimr2/rvautard/BCCORDEX/DRIAS/V1/pr/pr_rcp85_MOHC-HadGEM2-ES_RCA4_1971-2100.nc'

In [26]:
os.path.join(OUT_DIR,prefix+'.nc')

'/home/estimr2/nhempelmann/data/extremoscop/R20mm/rcp85/1/R20mm_rcp85_ICHEC-EC-EARTH_HIRHAM5_1971-2100.nc'

In [24]:
for nc in pr_files[0:1]:
    for indice in indices:
        for ugid in range(1,2):
            rcp = nc.split('_')[1]
            prefix = nc.replace('pr', indice).replace('.nc', '')
            # create foulder structure and define output dir:
            
            if not os.path.exists(os.path.join(OUT_PATH , indice ,rcp , str(ugid))):
                os.makedirs(os.path.join(OUT_PATH , indice ,rcp , str(ugid)))
            OUT_DIR = os.path.join(OUT_PATH , indice ,rcp , str(ugid))

In [29]:
range(1,18)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]